<a href="https://colab.research.google.com/github/uvconnects/AdvanceJavascript/blob/master/Prompt_engineeringClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28 tiktoken session-info --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.7 MB/s eta 0:00:00


In [ ]:
!pip install openai==0.28

In [ ]:
import openai # to wrok wth gpt-3.5-turbo model - this is our LLM
import json # to extract results from the LLM response
import tiktoken # to see background progress
import session_info # to get details on the colab session
import pandas as pd # to work with datasets
import time # to pause iterations - to account for rate limits

In [ ]:
session_info.show()

In [ ]:
%%writefile config.json
{
  "AZURE_OPENAI_KEY": "9fa7b77bc8c14575866c3bda9456f78e",
 "AZURE_OPENAI_ENDPOINT": "https://seven-n-name.openai.azure.com/",
 "AZURE_OPENAI_APITYPE": "azure",
 "AZURE_OPENAI_APIVERSION": "2023-05-15",
 "CHATGPT_MODEL":"mylml35turbo"
 }



Overwriting config.json


In [ ]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [ ]:
creds = json.loads(data)

In [ ]:
# Credentials to authenticate to the personalized Open AI model server
openai.api_key = creds["AZURE_OPENAI_KEY"]
openai.api_base = creds["AZURE_OPENAI_ENDPOINT"]
openai.api_type = creds["AZURE_OPENAI_APITYPE"]
openai.api_version = creds["AZURE_OPENAI_APIVERSION"]

# Deployment id of the ChatCompletion endpoint
chat_model_id = creds["CHATGPT_MODEL"]

In [ ]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    print(num_tokens)
    print('Please wait while the program is loading...')
    return num_tokens

In [ ]:
get_completion("What happen to kennedy")

'John F. Kennedy, the 35th President of the United States, was assassinated in Dallas, Texas on November 22, 1963. He was shot while riding in an open car during a motorcade. The assassination was a traumatic event for the nation and has been the subject of much speculation and conspiracy theories. Lee Harvey Oswald was arrested and charged with the assassination, but he was himself killed before he could stand trial.'

In [ ]:
df = pd.read_csv("patient_reviews.csv")

In [ ]:
df['patient_review_text'][0]

"John Smith - I recently had the privilege of consulting with Dr. Emily Roberts, a board-certified Family Physician, and I must say it was an exceptional experience. From start to finish, everything about my visit was top-notch, and I couldn't be more satisfied with the care I received. Dr. Roberts' credentials and professionalism were immediately evident. Her warm and welcoming demeanor put me at ease right away. She took the time to listen attentively to my concerns, showing genuine empathy and understanding. This made me feel like more than just a patient; I felt like a valued individual whose health truly mattered. What stood out to me the most was Dr. Roberts' comprehensive knowledge and expertise. She asked detailed questions to get a full understanding of my medical history, and her diagnosis and treatment recommendations were both thorough and easy to understand. It was evident that she had a deep understanding of the latest medical advancements and treatment options. Throughou

In [ ]:
def extract_review_details(text):

    prompt = f"""
    You are an assistant that supports a healthcare provider in analyzing patient reviews
    Your goal is to extract key information from the user message, including the patient's name, the doctor mentioned in the review,
    the review rating, a brief description of the review, and whether the patient expressed satisfaction with their appointment.
    Go through the user's feedback step by step, and generate a structured output for further analysis by the healthcare provider
    in the below format as a json

    "patient_name": <extract the patient’s first and last name from the corpus>,
    "consulting_doctor": <extract the doctor’s first and last name and credentials from the corpus>,
    "review_rating": <this has to be a number out of 5 points - if you cannot find a rating, output NULL>,
    "review_description": <summarize the review at most in 50 words>
    "satisfaction": <this has to be a TRUE or FALSE value - arrive at this conclusion using your own judgment>,
    "issue_tags": <in the case of a negative review or dissatisfaction, add tags which specify the area of dissatisfaction as comma separated values>

    Below is the review:
    ```{text}```

    """.format(text)
    #
    response = openai.ChatCompletion.create(
    engine=chat_model_id,
    messages = [{"role": "user", "content": prompt}],
    temperature=0,
    max_tokens=4096,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

  #  json_response = response.choices[0].message["content"]
   # data = json.loads(json_response)
    return response.choices[0].message["content"]
   # return pd.DataFrame(data, index=[0])

In [ ]:
#review_details_df = pd.DataFrame()
review_1 = df['patient_review_text'][0]
# review_1 = df['patient_review_text][0]
# print(extract_review)

In [ ]:
for i in df['patient_review_text']:
    df_response = extract_review_details(i)
    review_details_df = pd.concat([review_details_df, df_response])
    time.sleep(30) # rate limit is 60 seconds

In [ ]:
review_details_df

,patient_name,consulting_doctor,review_rating,review_description,satisfaction,issue_tags
0,John Smith,"Dr. Emily Roberts, Board-certified Family Phys...",5,"Exceptional experience with Dr. Emily Roberts,...",True,None
0,Sarah Johnson,Dr. David Smith,2,Sarah Johnson had a disappointing consultation...,False,"dissmissive, unprofessional, rushed, disintere..."
